## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from data_preprocessing import get_sparse_matrix, get_json

**Данные о товарах**

In [2]:
nom = pd.read_parquet('data/nomenclature.parquet')  # номенклатура

In [3]:
target_types_nom = {'Соки и сокосодержащие напитки', 'Снеки', 'Вода',
                    'Сладкие Уранированные напитки, холодный чай',
                    'Кофейные напитки с молоком', 'Энергетические напитки'}

target_ids = nom.sku_id[nom.sku_group.isin(target_types_nom)]

code_in = {indx:nom.sku_id.iloc[indx] for indx in nom.sku_id.index}
code_out = {nom.sku_id.iloc[indx]:indx for indx in nom.sku_id.index}

In [4]:
del nom

**Тренировочные данные для Item2Item**

In [5]:
train_sparse, _, _ = get_sparse_matrix('json_data_train', code_in)

**Тренировочные данные для модели Байеса**

In [6]:
train_known_sparse, train_known_ids, _ = get_sparse_matrix('json_data_known_clients', code_in)

**Валидационные данные**

In [7]:
subm = pd.read_parquet('data/transactions-for_submission.parquet')

In [8]:
subm_grouped_by_cheq = subm.groupby('cheque_id')

In [9]:
subm_groups_names = list(subm_grouped_by_cheq.indices.keys())

In [10]:
# get_json(subm_grouped_by_cheq, subm_groups_names, foldername='validation', n=20)

0

In [ ]:
del subm
del subm_grouped_by_cheq
del subm_groups_names

In [7]:
val_sparse, val_cl_ids, val_cheqs_ids = get_sparse_matrix('json_data_validation', code_in)

## Применение моделей

In [8]:
import implicit

**Item2Item**

In [9]:
from i2i import I2I

Обучение модели:

In [58]:
model_i2i = I2I(list(target_ids), code_in, code_out)

In [59]:
model_i2i.fit(train_sparse)

Применение модели:

In [60]:
from tqdm.notebook import tqdm

In [61]:
subm_i2i_res = {'cheque_id':[], 'sku_id':[], 'position':[]}

for cl_i in tqdm(range(val_sparse.shape[0])):
    pred_i = model_i2i.predict(val_sparse[cl_i])
    pos = 1
    for sku in pred_i:
        subm_i2i_res['cheque_id'].append(val_cheqs_ids[cl_i])
        subm_i2i_res['sku_id'].append(sku)
        subm_i2i_res['position'].append(pos)
        pos += 1
        
results_i2i = pd.DataFrame(subm_i2i_res, dtype=np.int64)

Сохранение результатов:

In [62]:
import os

In [63]:
# создание папки для сохранения результатов
if not 'results' in os.listdir():
        os.mkdir('results')

In [64]:
# results_i2i.to_parquet(path='results/submission-Skibin_Maxim_I2I_new')

**BI2I**

In [65]:
from bi2i import BI2I

Обучение модели:

In [66]:
model_bi2i = BI2I(train_known_ids, target_ids, code_in, code_out)

In [67]:
model_bi2i.fit(train_known_sparse, train_sparse)

Модель Байеса:



модель Item2Item:


Применение модели:

In [68]:
subm_bi2i_res = {'cheque_id':[], 'sku_id':[], 'position':[]}

for cl_i in tqdm(range(val_sparse.shape[0])):
    pred_i = model_bi2i.predict(val_sparse[cl_i], val_cl_ids[cl_i], 0.93)
    pos = 1
    for sku in pred_i:
        subm_bi2i_res['cheque_id'].append(val_cheqs_ids[cl_i])
        subm_bi2i_res['sku_id'].append(sku)
        subm_bi2i_res['position'].append(pos)
        pos += 1
        
results_bi2i = pd.DataFrame(subm_bi2i_res, dtype=np.int64)

Сохранение результатов:

In [69]:
# results_bi2i.to_parquet(path='results/submission-Skibin_Maxim_BI2I_new')